<img src="img/pandora2d_logo.png" width="500" height="500">

# Pandora2D : a coregistration framework

# Usage of step and ROI (Region of Interest) parameters in the configuration file

#### Imports and external functions

In [ ]:
import io
from pprint import pprint
from pathlib import Path
from IPython.display import Image, display
import numpy as np

In [ ]:
def plot_state_machine(pandora2d_machine):
    """
    Show the schemes of step of Pandora2D Machine
    """
    stream = io.BytesIO()
    try:
        pandora2d_machine.get_graph().draw(stream, prog='dot', format='png')
        display(Image(stream.getvalue()))
    except:
        print("It is not possible to show the graphic of the state machine. To solve it, please install graphviz on your system (apt-get install graphviz if operating in Linux) and install python package with pip insall graphviz")

In [ ]:
from snippets.utils import *

# Pandora2D execution options with state machine

#### Imports of pandora2d

In [ ]:
# Load pandora2d imports
from pandora2d import run 
from pandora2d.state_machine import Pandora2DMachine
from pandora2d.check_configuration import check_conf
from pandora2d.img_tools import get_roi_processing, create_datasets_from_inputs

#### Load and visualize input data 

Provide image path

In [ ]:
# Paths to left and right images
img_left_path = "data/left.tif"
img_right_path = "data/right.tif"

Provide output directory to write results

In [ ]:
output_dir = os.path.join(os.getcwd(),"output")
# If necessary, create output dir
Path(output_dir).mkdir(exist_ok=True,parents=True)

Convert input data to dataset

In [ ]:
input_config = {
    "left": {"img": img_left_path, "nodata": np.nan},
    "right": {"img": img_right_path, "nodata": np.nan},
    "col_disparity": {"init": 0, "range": 2},
    "row_disparity": {"init": 0, "range": 2},
}

In [ ]:
img_left, img_right = create_datasets_from_inputs(input_config=input_config)

Visualize input data

In [ ]:
fig = plt.figure(figsize=(10,10))
ax0 = fig.add_subplot(1,2,1)
ax0.imshow(img_left["im"].data)
plt.title("Left image")
ax1 = fig.add_subplot(1,2,2)
ax1.imshow(img_right["im"].data)
plt.title("Right image")

# Option 1 : Classic configuration file 

#### Instantiate the machine

In [ ]:
pandora2d_machine = Pandora2DMachine()

#### Define pipeline configuration

In [ ]:
user_cfg = {
    "input": {
        "left": {
            "img": img_left_path,
            "nodata": "NaN",
        },
        "right": {
            "img": img_right_path,
            "nodata": "NaN",
        },
        "col_disparity": {"init": 0, "range": 3},
        "row_disparity": {"init": 0, "range": 3},
    },
    "pipeline":{
        "matching_cost" : {
            "matching_cost_method": "zncc",
            "window_size": 7,
        },
        "disparity": {
            "disparity_method": "wta",
            "invalid_disparity": np.nan
        },
        "refinement" : {
            "refinement_method": "dichotomy",
            "iterations": 1,
            "filter": {"method": "bicubic"},
        }
    },
    "output": {
        "path": "outputs/usage_step_roi_config_option_1"
    },
}

#### Check the user configuration

In [ ]:
cfg = check_conf(user_cfg, pandora2d_machine)
pprint(cfg)

#### Prepare the machine 

In [ ]:
image_datasets = create_datasets_from_inputs(input_config=cfg["input"])

In [ ]:
pandora2d_machine.run_prepare(image_datasets.left, image_datasets.right, cfg)

#### Trigger all the steps of the machine at ones

In [ ]:
dataset, _ = run(pandora2d_machine, image_datasets.left, image_datasets.right, cfg)

#### Visualize output disparity maps

Row disparity map

In [ ]:
plot_image(dataset["row_map"].data, "Row refined disparity map", output_dir, cmap=pandora_cmap())

Column disparity map

In [ ]:
plot_image(dataset["col_map"].data, "Columns refined disparity map", output_dir, cmap=pandora_cmap())

# Option 2 : Add a step parameter in the matching cost computation

It is possible to add a "step" parameter in the matching cost computation in order not to perform the calculation for each pixel. 

#### Instantiate the machine

In [ ]:
pandora2d_machine = Pandora2DMachine()

#### Define pipeline configuration

In [ ]:
user_cfg = {
    "input": {
        "left": {
            "img": img_left_path,
            "nodata": "NaN",
        },
        "right": {
            "img": img_right_path,
            "nodata": "NaN",
        },
        "col_disparity": {"init": 0, "range": 3},
        "row_disparity": {"init": 0, "range": 3},
    },
    "pipeline":{
        "matching_cost" : {
            "matching_cost_method": "zncc",
            "window_size": 7,
            "step" : [5,5]
        },
        "disparity": {
            "disparity_method": "wta",
            "invalid_disparity": np.nan
        },
        "refinement" : {
            "refinement_method": "dichotomy",
            "iterations": 1,
            "filter": {"method": "bicubic"},
        }
    },
    "output": {
        "path": "outputs/usage_step_roi_config_option_2"
    },
}

#### Check the user configuration

In [ ]:
cfg = check_conf(user_cfg, pandora2d_machine)
pprint(cfg)

#### Prepare the machine 

In [ ]:
image_datasets = create_datasets_from_inputs(input_config=cfg["input"])

In [ ]:
pandora2d_machine.run_prepare(image_datasets.left, image_datasets.right, cfg)

#### Trigger all the steps of the machine at ones

In [ ]:
dataset, _ = run(pandora2d_machine, image_datasets.left, image_datasets.right, cfg)

#### Visualize output disparity maps

Row disparity map

In [ ]:
plot_image(dataset["row_map"].data, "Row refined disparity map", output_dir, cmap=pandora_cmap())

Column disparity map

In [ ]:
plot_image(dataset["col_map"].data, "Columns refined disparity map", output_dir, cmap=pandora_cmap())

# Option 3 : Add a ROI 

It is possible to work on only one section of the image with an ROI. For this, the user can specify the area he wants in the configuration file. 

#### Instantiate the machine

In [ ]:
pandora2d_machine = Pandora2DMachine()

#### Define pipeline configuration

In [ ]:
user_cfg = {
    "input": {
        "left": {
            "img": img_left_path,
            "nodata": "NaN",
        },
        "right": {
            "img": img_right_path,
            "nodata": "NaN",
        },
        "col_disparity": {"init": 0, "range": 3},
        "row_disparity": {"init": 0, "range": 3},
    },
    "ROI":{
        "col": {"first": 10, "last": 100},
        "row": {"first": 10, "last": 100}
    },
    "pipeline":{
        "matching_cost" : {
            "matching_cost_method": "zncc",
            "window_size": 7,
        },
        "disparity": {
            "disparity_method": "wta",
            "invalid_disparity": np.nan
        },
        "refinement" : {
            "refinement_method": "dichotomy",
            "iterations": 1,
            "filter": {"method": "bicubic"},
        }
    },
    "output": {
        "path": "outputs/usage_step_roi_config_option_3"
    },
}

#### Check the user configuration

In [ ]:
cfg = check_conf(user_cfg, pandora2d_machine)
pprint(cfg)

#### Add margins to the ROI to compute all the disparities in the col_disparity and row_disparity intervals

In [ ]:
cfg["ROI"]["margins"] = pandora2d_machine.margins_img.global_margins.astuple()
roi = get_roi_processing(cfg["ROI"], cfg["input"]["col_disparity"], cfg["input"]["row_disparity"])

#### Prepare the machine : Only the region of interest (ROI) is read and returned

In [ ]:
image_datasets = create_datasets_from_inputs(input_config=cfg["input"], roi=roi)

In [ ]:
pandora2d_machine.run_prepare(image_datasets.left, image_datasets.right, cfg)

#### Trigger all the steps of the machine at ones

In [ ]:
dataset, _ = run(pandora2d_machine, image_datasets.left, image_datasets.right, cfg)

#### Visualize output disparity maps

Processing margins are included in the disparity map view below. These can be removed by calling the method pandora2d.img_tools.remove_roi_margins()

Row disparity map

In [ ]:
plot_image(dataset["row_map"].data, "Row refined disparity map", output_dir, cmap=pandora_cmap())

Column disparity map

In [ ]:
plot_image(dataset["col_map"].data, "Columns refined disparity map", output_dir, cmap=pandora_cmap())